## The Command pattern

A pattern that makes state changes in other objects.
The Command pattern generally involves a hierarchy of classes that each do
something. A Core class can create a command (or a sequence of commands) to carry
out actions.
![](uml/command_pattern.png)


### A Command example



In [1]:
from __future__ import annotations
import abc
import re
import random
from typing import cast, Optional, Union, Sequence

random.seed(42)


class Adjustment(abc.ABC):
    def __init__(self, amount: int) -> None:
        self.amount = amount
        
    @abc.abstractmethod
    def apply(self, dice: "Dice") -> None:
        ...
        
    def __repr__(self) -> str:
        return f"{type(self).__name__}({self.amount})"   
    
    
class Roll(Adjustment):
    def __init__(self, n: int, d: int) -> None:
        self.n = n
        self.d = d
    
    def apply(self, dice: "Dice") -> None:
        dice.dice = sorted(random.randint(1, self.d) for _ in range(self.n))
        dice.modifier = 0
        
    def __repr__(self) -> str:
        return f"{type(self).__name__}(n={self.n}, d={self.d})"
    

class Drop(Adjustment):
    def apply(self, dice: "Dice") -> None:
        dice.dice = dice.dice[self.amount :]
    
    
class Keep(Adjustment):    
    def apply(self, dice: "Dice") -> None:
        dice.dice = dice.dice[: self.amount]
    
    
class Plus(Adjustment):
    def apply(self, dice: "Dice") -> None:
        dice.modifier += self.amount
    

class Minus(Adjustment):
    def apply(self, dice: "Dice") -> None:
        dice.modifier -= self.amount
    
    
class Dice:
    """
    >>> import random
    >>> random.seed(0)
    >>> d = Dice(5, 6, Drop(2), Keep(2), Plus(100), Minus(50))
    >>> r = d.roll()
    >>> print(d.dice)
    [4, 4]
    >>> print(d.modifier)
    50
    """
    def __init__(self, n: int, d: int, *adj: Adjustment) -> None:
        self.adjusments = [cast(Adjustment, Roll(n, d))] + list(adj)
        self.dice: list[int]
        self.modifier: int
            
    def roll(self) -> int:
        for a in self.adjusments:
            a.apply(self)
        return sum(self.dice) + self.modifier
    
    @classmethod
    def from_text(cls, dice_text: str) -> "Dice":
        dice_pattern = re.compile(r"(?P<n>\d*)d(?P<d>\d+)(?P<a>[dk+-]\d+)*")
        adjustment_pattern = re.compile(r"([dk+-])(\d+)")
        adj_class: dict[str, type[Adjustment]] = {
            "d": Drop,
            "k": Keep,
            "+": Plus,
            "-": Minus,
        }
        if (dice_match := dice_pattern.match(dice_text)) is None:
            raise ValueError(f"Error in {dice_text!r}")

        n = int(dice_match.group("n")) if dice_match.group("n") else 1
        d = int(dice_match.group("d"))
        adjustment_matches = adjustment_pattern.finditer(dice_match.group("a") or "")
        adjustments = [
            adj_class[a.group(1)](int(a.group(2))) for a in adjustment_matches
        ]
        return cls(n, d, *adjustments)
        
    def __repr__(self) -> str:
        return f"{type(self).__name__}({', '.join(str(i) for i in self.adjusments)})"

    

d1 = Dice.from_text("4d6k3")
print(repr(d1))
print(d1.roll(), end='\n\n')

d2 = Dice.from_text("3d7+11")
print(repr(d2))
print(d2.roll(), end='\n\n')

d3 = Dice.from_text("3d7-1")
print(repr(d3))
print(d3.roll(), end='\n\n')

d4 = Dice.from_text("3d7d2")
print(repr(d4))
print(d4.roll(), end='\n\n')

Dice(Roll(n=4, d=6), Keep(3))
8

Dice(Roll(n=3, d=7), Plus(11))
18

Dice(Roll(n=3, d=7), Minus(1))
8

Dice(Roll(n=3, d=7), Drop(2))
6



In [2]:
if __name__ == '__main__':        
    import doctest
    import subprocess
    name = "05-The Command pattern"
    doctest.testmod(verbose=False)
    subprocess.run(f'jupyter nbconvert --to script --output test "{name}"', shell=True)
    std_out = subprocess.run('mypy --strict test.py', capture_output=True, shell=True).stdout
    print(std_out.decode('ascii'))

[NbConvertApp] Converting notebook 05-The Command pattern.ipynb to script
[NbConvertApp] Writing 3874 bytes to test.py


Success: no issues found in 1 source file

